# Step1： 设置基本参数

In [ ]:
import os
from dotenv import load_dotenv
ROOT_DIR= os.path.dirname(os.getcwd())
print(ROOT_DIR)
# 加载环境变量
load_dotenv()
print(os.getenv('ZISHU_API_KEY'))
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
print(api_key)
# base_url = "http://101.132.164.17:8000/v1"
base_url = "https://open.bigmodel.cn/api/paas/v4"
chat_model = "glm-4-flash"
emb_model = "embedding-2"






c:\Users\zata\Desktop\wow\01-wow-agent
9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR
9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR


# Step2 创建OurLLM的实例

In [ ]:
from openai import OpenAI
from pydantic import Field  # 导入Field，用于Pydantic模型中定义字段的元数据
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    LLMMetadata,
)
from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.llms.callbacks import llm_completion_callback
from typing import List, Any, Generator

# 定义OurLLM类，继承自CustomLLM基类
class OurLLM(CustomLLM):
    # 定义类属性，使用Field为Pydantic模型字段添加元数据
    api_key: str = Field(default=api_key)  # OpenAI API密钥
    base_url: str = Field(default=base_url)  # OpenAI API的基础URL
    model_name: str = Field(default=chat_model)  # 使用的模型名称
    client: OpenAI = Field(default=None, exclude=True)  # OpenAI客户端实例，exclude=True表示不包含在序列化中

    # 初始化方法，接收api_key、base_url、model_name等参数
    def __init__(self, api_key: str, base_url: str, model_name: str = chat_model, **data: Any):
        super().__init__(**data)  # 调用父类的初始化方法
        self.api_key = api_key  # 设置api_key
        self.base_url = base_url  # 设置base_url
        self.model_name = model_name  # 设置model_name
        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url)  # 初始化OpenAI客户端

    # 定义metadata属性，返回LLM的元数据
    # 这里的property 装饰器表示这是一个只读属性，不能修改
    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            model_name=self.model_name,  # 返回模型名称
        )

    # 实现complete方法，用于处理同步的文本生成请求
    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        # 调用OpenAI API生成文本
        response = self.client.chat.completions.create(model=self.model_name, messages=[{"role": "user", "content": prompt}])
        print(response)
        # 检查响应中是否有choices字段，并提取生成的文本
        if hasattr(response, 'choices') and len(response.choices) > 0:
            response_text = response.choices[0].message.content
            return CompletionResponse(text=response_text)  # 返回生成的文本
        else:
            raise Exception(f"Unexpected response format: {response}")  # 如果响应格式不符合预期，抛出异常

    # 实现stream_complete方法，用于处理流式文本生成请求
    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> Generator[CompletionResponse, None, None]:
        # 调用OpenAI API进行流式文本生成
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}],
            stream=True # 定义stream=True，以启用流式响应
        )
        print(response)

        try:
            # 遍历流式响应的每个chunk，提取生成的文本
            for chunk in response:
                chunk_message = chunk.choices[0].delta
                if not chunk_message.content:  # 如果chunk中没有内容，跳过
                    continue
                content = chunk_message.content
                yield CompletionResponse(text=content, delta=content)  # 返回生成的文本和增量内容

        except Exception as e:
            raise Exception(f"Unexpected response format: {e}")  # 如果发生异常，抛出异常

# 创建OurLLM的实例，传入api_key、base_url

注意看上面的代码，不知道你有没有疑惑：上面实现了两种文本处理请求，一种是用于处理同步的文本生成请求，一种是用于处理流式的文本生成请求。它们之间的区别是什么?

简单来说，同步的文本生成请求会等待模型生成完所有文本，然后再返回给用户。而流式的文本生成请求会实时返回模型生成的文本，用户可以实时看到模型正在生成文本。

详细可以可以看Explanation_Doc.md

In [6]:
llm = OurLLM(api_key=api_key, base_url=base_url, model_name=chat_model)
response = llm.complete("你是谁？")
for chunk in response:
    print(chunk, end="", flush=True)

ChatCompletion(id='2025021500065953ef0570e768407f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='我是一个人工智能助手，名叫 ChatGLM，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739549220, model='glm-4-flash', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=8, total_tokens=54, completion_tokens_details=None, prompt_tokens_details=None), request_id='2025021500065953ef0570e768407f')
('text', '我是一个人工智能助手，名叫 ChatGLM，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。')('additional_kwargs', {})('raw', None)('logprobs', None)('delta', None)

In [7]:
llm = OurLLM(api_key=api_key, base_url=base_url, model_name=chat_model)
response = llm.stream_complete("你是谁？")
for chunk in response:
    print(chunk, end="", flush=True)

我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。

# Step3：ReActAgent 和 FunctionTool  使用 
 
ReActAgent.from_tools 工作流程解释 可见 Explanation_Doc.md

## 实现简单数学运算的智能代理

In [8]:
import sys
import os

# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "..")))

from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool


def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


def main():

    multiply_tool = FunctionTool.from_defaults(fn=multiply)
    add_tool = FunctionTool.from_defaults(fn=add)

    # 创建ReActAgent实例
    agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

    response = agent.chat("20+（2*4）等于多少？使用工具计算每一步")

    print(response)


if __name__ == "__main__":
    main()


> Running step 2c868f0c-b18f-4f55-aa38-57dadddae21b. Step input: 20+（2*4）等于多少？使用工具计算每一步
ChatCompletion(id='20250215000718f70bb9a88642493f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thought: The user is asking to calculate an expression with addition and multiplication. I need to use the appropriate tools to calculate each step of the expression.\nAction: multiply\nAction Input: {"a": 2, "b": 4}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739549239, model='glm-4-flash', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=51, prompt_tokens=592, total_tokens=643, completion_tokens_details=None, prompt_tokens_details=None), request_id='20250215000718f70bb9a88642493f')
Thought: The user is asking to calculate an expression with addition and multiplication. I need to use the appropriate tools to calculate each step of the expression.
Acti

## 实现天气查询和数学运算的智能代理

In [9]:
def get_weather(city: str) -> int:
    """
    Gets the weather temperature of a specified city.

    Args:
    city (str): The name or abbreviation of the city.

    Returns:
    int: The temperature of the city. Returns 20 for 'NY' (New York),
         30 for 'BJ' (Beijing), and -1 for unknown cities.
    """

    # Convert the input city to uppercase to handle case-insensitive comparisons
    city = city.upper()

    # Check if the city is New York ('NY')
    if city == "NY":
        return 20  # Return 20°C for New York

    # Check if the city is Beijing ('BJ')
    elif city == "BJ":
        return 30  # Return 30°C for Beijing

    # If the city is neither 'NY' nor 'BJ', return -1 to indicate unknown city
    else:
        return -1

weather_tool = FunctionTool.from_defaults(fn=get_weather)

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

agent = ReActAgent.from_tools([multiply_tool, add_tool, weather_tool], llm=llm, verbose=True)

response = agent.chat("纽约天气怎么样?")


> Running step dc4dcabc-f68d-47ac-bba8-7ce1d5b380cc. Step input: 纽约天气怎么样?
ChatCompletion(id='20250215001029d83f9f9ade844ead', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thought: The user is asking for the weather in New York. I need to use a tool to help me answer the question.\nAction: get_weather\nAction Input: {"city": "NY"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739549430, model='glm-4-flash', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=42, prompt_tokens=706, total_tokens=748, completion_tokens_details=None, prompt_tokens_details=None), request_id='20250215001029d83f9f9ade844ead')
Thought: The user is asking for the weather in New York. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'city': 'NY'}
Observation: 20
> Running step a151a75b-343c-4c74-b2c1-ab2181a1f6c7. Step input: N

# Step5： 数据库 对话agent 


## 创建一个sqlite 数据库 并插入数据

sqlite数据库可以用上sqlite3库操作，如果查看的话 推荐可以使用SQLiteStudio或者sqlitebrowser这样的可视化工具进行查看

In [6]:
import sqlite3
# 示例数据
data = [
    ["专利部",22],
    ["商标部",25],
]


# 创建数据库
sqllite_path = 'llmdb.db'
con = sqlite3.connect(sqllite_path)

# 如果表不存在则创建表
sql = """
CREATE TABLE IF NOT EXISTS section_stats (
  部门 TEXT DEFAULT NULL,
  人数 INTEGER DEFAULT NULL
);
"""
c = con.cursor() # 创建游标
cursor = c.execute(sql) # 执行sql语句
c.close() # 关闭游标
con.close()  # 关闭数据库连接

con = sqlite3.connect(sqllite_path) # 创建数据库连接
c = con.cursor() # 创建游标


# 将数据插入数据库
for item in data:
    sql = """
    INSERT INTO section_stats (部门,人数) 
    values('%s','%d')
    """%(item[0],item[1])
    c.execute(sql)  # 执行sql语句
    con.commit() # 提交事务
c.close() # 关闭游标
con.close() # 关闭数据库连接

## 使用 智谱ai的api的llm和embedding模型

In [ ]:
import os
from dotenv import load_dotenv
ROOT_DIR= os.path.dirname(os.getcwd())
print(ROOT_DIR)
# 加载环境变量
load_dotenv()
print(os.getenv('ZISHU_API_KEY'))
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
chat_model = "glm-4-flash"
emb_model = "embedding-2"



# -----------------------------------------------------
# 前面和之前一样，不过下面的llm创建不一样了,这里不再使用openai库，而是转而使用llama_index库


# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI
llm = ZhipuAI(
    api_key = api_key,
    model = chat_model,
)
#

# 测试对话模型
response = llm.complete("你是谁？")
print(response)

# 配置嵌入模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding
embedding = ZhipuAIEmbedding(
    api_key = api_key,
    model = emb_model,
)


# 测试嵌入模型
emb = embedding.get_text_embedding("你好呀呀")
print(emb)
len(emb), type(emb)

c:\Users\zata\Desktop\wow\01-wow-agent
9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR
我是一个人工智能助手，基于大型语言模型进行设计的。我的目标是帮助用户回答问题、提供信息和解决各种问题。
[-0.03837227, -0.00019120687, -0.010714491, -0.015073115, 0.016608356, -0.050502937, -0.025533188, -0.027715547, 0.021502344, 0.032557108, -0.03067428, -0.02086279, 0.0046875053, 0.034516953, -0.042270742, 0.041547254, -0.019662227, 0.026853487, -0.005376031, 0.009594618, -0.018981915, -0.013965981, -0.018312378, 0.0031198212, 0.04439648, -0.015677191, 0.009684721, -0.06881334, -0.047387555, -0.0016778187, 0.03813079, 0.0013801784, -0.04599552, 0.019898636, 0.018735, 0.027104413, 0.018217556, 0.042128537, 0.04531775, 0.03019625, -0.023391131, -0.01604703, -0.015947886, -0.0049376804, 0.044406954, -0.027735537, 0.055833668, 0.009265803, -0.018208269, -0.026014157, -0.016125116, -0.01624468, 0.02994953, 0.0187377, 0.020882482, 0.0121635, -0.040657587, -0.0013120841, -0.01602421, 0.048642263, -0.02346143, 0.0066600163, -0.0068642013, 0.00019221354, 0

(1024, list)

In [3]:
from llama_index.core.agent import ReActAgent  
from llama_index.core.tools import FunctionTool  
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings  
from llama_index.core.tools import QueryEngineTool   
from llama_index.core import SQLDatabase  
from llama_index.core.query_engine import NLSQLTableQueryEngine  
from sqlalchemy import create_engine, select  


# 配置默认大模型  
Settings.llm = llm
Settings.embed_model = embedding

In [ ]:
## 创建数据库查询引擎  
engine = create_engine("sqlite:///llmdb.db")  

sql_database = SQLDatabase(engine, include_tables=["section_stats"])  
query_engine = NLSQLTableQueryEngine(  
    sql_database=sql_database,   
    tables=["section_stats"],   
    llm=Settings.llm  
)

In [5]:
# 创建工具函数  
def multiply(a: float, b: float) -> float:  
    """将两个数字相乘并返回乘积。"""  
    return a * b  

multiply_tool = FunctionTool.from_defaults(fn=multiply)  

def add(a: float, b: float) -> float:  
    """将两个数字相加并返回它们的和。"""  
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

# 把数据库查询引擎封装到工具函数对象中  
staff_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="section_staff",
    description="查询部门的人数。"  
)


In [6]:
# 构建ReActAgent，可以加很多函数，在这里只加了加法函数和部门人数查询函数。
agent = ReActAgent.from_tools([add_tool, staff_tool], verbose=True)  
# 通过agent给出指令
response = agent.chat("请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！")  

> Running step 6075ac0d-9bec-42a1-a549-154287c655ec. Step input: 请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: section_staff
Action Input: {'input': '专利部'}
Observation: 专利部的人数是22人。
> Running step 80eb16b0-b2eb-4152-9364-97b4108b25bc. Step input: None
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: section_staff
Action Input: {'input': '商标部'}
Observation: 商标部的人数为25人。
> Running step 3b62ed70-cbe3-48b8-8d32-8e1db84ba8cd. Step input: None
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 22, 'b': 25}
Observation: 47
> Running step eb87cbaf-f5da-491f-aaf4-b5169b18021a. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 专利部和商标部的人数总和为47人。

In [7]:
# 构建ReActAgent，可以加很多函数，在这里只加了加法函数和部门人数查询函数。
agent = ReActAgent.from_tools([add_tool, staff_tool], verbose=True)  
# 通过agent给出指令
response = agent.chat("请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！")  

> Running step de2d1179-6d13-48ce-9335-53770ff79240. Step input: 请从数据库表中获取`专利部`和`商标部`的人数，并将这两个部门的人数相加！
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: section_staff
Action Input: {'input': '专利部'}
Observation: 专利部的员工人数为22人。
> Running step 8fc40189-0397-4142-83c7-89e763bd3cf3. Step input: None
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: section_staff
Action Input: {'input': '商标部'}
Observation: The response to the query would be based on the actual data found in the "section_stats" table within the database. Since I cannot execute the SQL query due to the lack of a connected database, I will simulate a possible response based on the expected outcome:

Response: The "商标部" has [X] staff members, where [X] would be the number of people listed in the "商标部" section in the database. Please refer to the latest database records for the precise number.
> Runni